In [ ]:
#CASE 1

from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tsai.all import *

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

def training_loss(pred, target):
    return F.cross_entropy(pred, target)

def train_and_extract_embeddings_in_batches(dsid, hidden_size, device, batch_size=16, save_path='train_embeddings.npy', val_save_path='val_embeddings.npy'):

    X_train, y_train, X_test, y_test = get_UCR_data(dsid, return_split=True)

    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

    print(f"X_train_split shape: {X_train_split.shape}, X_val_split shape: {X_val_split.shape}")

    tfms = [None, [Categorize()]]  
    dsets_train = TSDatasets(X_train_split, y_train_split, tfms=tfms, splits=None, inplace=True)
    #dsets_val = TSDatasets(X_val_split, y_train_split, tfms=tfms, splits=None, inplace=True)
    dls_train = TSDataLoaders.from_dsets(dsets_train.train, dsets_train.train, bs=batch_size, batch_tfms=[TSStandardize()], num_workers=0)
    #dls_val = TSDataLoaders.from_dsets(dsets_val.train, dsets_val.train, bs=batch_size, batch_tfms=[TSStandardize()], num_workers=0)

    c_in = dls_train.vars 
    c_out = dls_train.c 

    model = CustomCNN(c_in, c_out).to(device)
    learn = Learner(dls_train, model, loss_func=training_loss, metrics=accuracy) 
    learn.fit_one_cycle(25, 0.002)  

    model.eval()

    def extract_embeddings(X_split, save_path):
        X_split_tensor = torch.tensor(X_split).float().to(device)
        print(f"X_split_tensor shape before batching: {X_split_tensor.shape}")
        all_embeddings = []
        for i in range(0, len(X_split_tensor), batch_size):
            batch_X = X_split_tensor[i:i+batch_size]
            print(f"Batch_X shape before CNN: {batch_X.shape}")
            with torch.no_grad():
                _, batch_embeddings = model(batch_X, return_embeddings=True)  # Extract embeddings
            all_embeddings.append(batch_embeddings.cpu().numpy())
        all_embeddings = np.concatenate(all_embeddings, axis=0)
        np.save(save_path, all_embeddings)

    extract_embeddings(X_train_split, save_path)

    extract_embeddings(X_val_split, val_save_path)

dataset_ids = ['Libras', 'RacketSports', 'NATOPS', 'UWaveGestureLibrary', 'Cricket', 'Ering', 'BasicMotions', 'Epilepsy']
hidden_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Ensure the correct device (GPU or CPU)

train_embeddings_dict = {}
val_embeddings_dict = {}

for dsid in dataset_ids:
    train_save_path = f'train_embeddings_{dsid}.npy'
    val_save_path = f'val_embeddings_{dsid}.npy'
    train_and_extract_embeddings_in_batches(dsid, hidden_size, device, batch_size=16, save_path=train_save_path, val_save_path=val_save_path)

    train_embeddings_dict[dsid] = np.load(train_save_path)
    val_embeddings_dict[dsid] = np.load(val_save_path)

normalized_train_embeddings_dict = {dsid: normalize(train_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}
normalized_val_embeddings_dict = {dsid: normalize(val_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}

for dsid1, dsid2 in itertools.combinations(dataset_ids, 2):

    train_embeddings1_normalized = normalized_train_embeddings_dict[dsid1]
    train_embeddings2_normalized = normalized_train_embeddings_dict[dsid2]
    train_similarity_matrix = cosine_similarity(train_embeddings1_normalized, train_embeddings2_normalized)
    avg_train_similarity = np.mean(train_similarity_matrix)
    print(f"Average train set similarity between {dsid1} and {dsid2}: {avg_train_similarity}")

    val_embeddings1_normalized = normalized_val_embeddings_dict[dsid1]
    val_embeddings2_normalized = normalized_val_embeddings_dict[dsid2]
    val_similarity_matrix = cosine_similarity(val_embeddings1_normalized, val_embeddings2_normalized)
    avg_val_similarity = np.mean(val_similarity_matrix)
    print(f"Average validation set similarity between {dsid1} and {dsid2}: {avg_val_similarity}")

In [ ]:
#CASE 2 

import random
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tsai.all import *
import numpy as np
from art.attacks.evasion import (
    BasicIterativeMethod, DeepFool, CarliniL2Method, MomentumIterativeMethod,
    ElasticNet, AutoProjectedGradientDescent, FastGradientMethod, ZooAttack, BoundaryAttack
)
from art.estimators.classification import PyTorchClassifier
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

# Random number generator to choose the attack (0-9)
def random_attack_choice():
    return random.randint(0, 9)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

class CustomCNN(nn.Module):
    def __init__(self, input_size, output_size, num_filters=64, kernel_size=3, dropout=0.5):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_size)
        self.fc1 = nn.Linear(num_filters, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, return_embeddings=False):
        x = x.permute(0, 1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.adaptive_max_pool1d(x, 1).squeeze(2)

        x = self.dropout(x)
        final_output = self.fc1(x)

        if return_embeddings:
            embeddings = x
            return final_output, embeddings
        else:
            return final_output

def training_loss(pred, target):
    return F.cross_entropy(pred, target)

def create_art_classifier(model, input_shape, nb_classes, device):
    classifier = PyTorchClassifier(
        model=model,
        loss=training_loss,
        input_shape=input_shape,
        nb_classes=nb_classes,
        optimizer=torch.optim.Adam(model.parameters(), lr=0.002),
        device_type=device.type
    )
    return classifier

def apply_attack(X, classifier, attack_number):
    attack_number = 8
    if attack_number == 0:
        attack = BasicIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 1:
        attack = DeepFool(classifier=classifier)
    elif attack_number == 2:
        attack = CarliniL2Method(classifier=classifier)
    elif attack_number == 3:
        attack = MomentumIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 4:
        attack = ElasticNet(classifier=classifier)
    elif attack_number == 5:
        attack = AutoProjectedGradientDescent(estimator=classifier, eps=0.1)
    elif attack_number == 6:
        attack = FastGradientMethod(estimator=classifier, eps=0.1)
    elif attack_number == 7:
        attack = ZooAttack(classifier=classifier, confidence=0.5, targeted=False)
    elif attack_number == 8:
        attack = BoundaryAttack(estimator=classifier, targeted=False)
    else:
        return X
    return attack.generate(X)

def train_and_attack_validation(dsid, hidden_size, device, attack_number, batch_size=16, save_path='train_embeddings.npy', attacked_val_save_path='attacked_val_embeddings.npy'):

    X_train, y_train, X_test, y_test = get_UCR_data(dsid, return_split=True)

    #X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.4, stratify=y_train, shuffle=False)
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
    print(f"X_train_split shape: {X_train_split.shape}, X_val_split shape: {X_val_split.shape}")

    tfms = [None, [Categorize()]]
    dsets_train = TSDatasets(X_train_split, y_train_split, tfms=tfms, splits=None, inplace=True)
    dls_train = TSDataLoaders.from_dsets(dsets_train.train, dsets_train.train, bs=batch_size,shuffle=False,batch_tfms=[TSStandardize()], num_workers=0)

    c_in = dls_train.vars
    c_out = dls_train.c

    model = CustomCNN(c_in, c_out).to(device)
    learn = Learner(dls_train, model, loss_func=training_loss, metrics=accuracy)
    learn.fit_one_cycle(25, 0.002)

    model.eval()

    classifier = create_art_classifier(model, input_shape=(c_in, X_train_split.shape[2]), nb_classes=c_out, device=device)

    print(f"Applying attack {attack_number} to the validation set")
    attacked_X_val_split = apply_attack(X_val_split, classifier, attack_number)

    def extract_embeddings(X_split, save_path):
        X_split_tensor = torch.tensor(X_split).float().to(device)
        all_embeddings = []
        for i in range(0, len(X_split_tensor), batch_size):
            batch_X = X_split_tensor[i:i+batch_size]
            with torch.no_grad():
                _, batch_embeddings = model(batch_X, return_embeddings=True)
            all_embeddings.append(batch_embeddings.cpu().numpy())
        all_embeddings = np.concatenate(all_embeddings, axis=0)
        np.save(save_path, all_embeddings)

    extract_embeddings(X_train_split, save_path)

    extract_embeddings(attacked_X_val_split, attacked_val_save_path)

dataset_ids = ['RacketSports', 'NATOPS', 'UWaveGestureLibrary', 'Cricket', 'Ering', 'BasicMotions', 'Epilepsy']
hidden_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

attack_number = 2
print(f"Chosen attack number: {attack_number}")

train_embeddings_dict = {}
attacked_val_embeddings_dict = {}

for dsid in dataset_ids:
    train_save_path = f'train_embeddings_{dsid}.npy'
    attacked_val_save_path = f'attacked_val_embeddings_{dsid}.npy'
    train_and_attack_validation(dsid, hidden_size, device, attack_number, batch_size=16, save_path=train_save_path, attacked_val_save_path=attacked_val_save_path)

    train_embeddings_dict[dsid] = np.load(train_save_path)
    attacked_val_embeddings_dict[dsid] = np.load(attacked_val_save_path)

normalized_train_embeddings_dict = {dsid: normalize(train_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}
normalized_attacked_val_embeddings_dict = {dsid: normalize(attacked_val_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}

for dsid1, dsid2 in itertools.combinations(dataset_ids, 2):

    train_embeddings1_normalized = normalized_train_embeddings_dict[dsid1]
    train_embeddings2_normalized = normalized_train_embeddings_dict[dsid2]
    train_similarity_matrix = cosine_similarity(train_embeddings1_normalized, train_embeddings2_normalized)
    avg_train_similarity = np.mean(train_similarity_matrix)
    print(f"Average train set similarity between {dsid1} and {dsid2}: {avg_train_similarity}")

    attacked_val_embeddings1_normalized = normalized_attacked_val_embeddings_dict[dsid1]
    attacked_val_embeddings2_normalized = normalized_attacked_val_embeddings_dict[dsid2]
    attacked_val_similarity_matrix = cosine_similarity(attacked_val_embeddings1_normalized, attacked_val_embeddings2_normalized)
    avg_attacked_val_similarity = np.mean(attacked_val_similarity_matrix)
    print(f"Average attacked validation set similarity between {dsid1} and {dsid2}: {avg_attacked_val_similarity}")

In [ ]:
#CASE 3
import random
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tsai.all import *
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from art.attacks.evasion import (
    BasicIterativeMethod, DeepFool, CarliniL2Method, MomentumIterativeMethod,
    ElasticNet, AutoProjectedGradientDescent, FastGradientMethod, ZooAttack, BoundaryAttack
)
from art.estimators.classification import PyTorchClassifier


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

class CustomCNN(nn.Module):
    def __init__(self, input_size, output_size, num_filters=64, kernel_size=3, dropout=0.5):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_size)
        self.fc1 = nn.Linear(num_filters, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, return_embeddings=False):
        x = x.permute(0, 1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.adaptive_max_pool1d(x, 1).squeeze(2)
        x = self.dropout(x)
        final_output = self.fc1(x)

        if return_embeddings:
            embeddings = x
            return final_output, embeddings
        else:
            return final_output

def training_loss(pred, target):
    return F.cross_entropy(pred, target)

def create_art_classifier(model, input_shape, nb_classes, device):
    classifier = PyTorchClassifier(
        model=model,
        loss=training_loss,
        input_shape=input_shape,
        nb_classes=nb_classes,
        optimizer=torch.optim.Adam(model.parameters(), lr=0.002),
        device_type=device.type
    )
    return classifier

def apply_attack(X, classifier, attack_number):
    if attack_number == 0:
        attack = BasicIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 1:
        attack = DeepFool(classifier=classifier)
    elif attack_number == 2:
        attack = CarliniL2Method(classifier=classifier)
    elif attack_number == 3:
        attack = MomentumIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 4:
        attack = ElasticNet(classifier=classifier)
    elif attack_number == 5:
        attack = AutoProjectedGradientDescent(estimator=classifier, eps=0.1)
    elif attack_number == 6:
        attack = FastGradientMethod(estimator=classifier, eps=0.1)
    elif attack_number == 7:
        attack = ZooAttack(classifier=classifier, confidence=0.5, targeted=False)
    elif attack_number == 8:
        attack = BoundaryAttack(estimator=classifier, targeted=False)
    else:
        return X
    return attack.generate(X)

def apply_attack_decision(X_split, classifier, attack_decisions):
    attacked_pieces = []
    for piece, (attack_flag, attack_number) in zip(X_split, attack_decisions):
        if attack_flag == 1:
            attacked_piece = apply_attack(piece, classifier, attack_number)
        else:
            attacked_piece = piece
        attacked_pieces.append(attacked_piece)
    return np.concatenate(attacked_pieces, axis=0)

def split_data(X, num_pieces=5):
    splits = np.array_split(X, num_pieces)
    return splits

def generate_attack_decisions(num_pieces=5):
    attack_decisions = []
    for _ in range(num_pieces):
        attack_flag = random.randint(0, 1)
        attack_number = random.randint(0, 9) if attack_flag == 1 else None
        attack_decisions.append((attack_flag, attack_number))
    return attack_decisions

def train_and_attack_validation(dsid, hidden_size, device, attack_decisions, batch_size=16, save_path='train_embeddings.npy', attacked_val_save_path='attacked_val_embeddings.npy'):

    X_train, y_train, X_test, y_test = get_UCR_data(dsid, return_split=True)

    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

    print(f"X_train_split shape: {X_train_split.shape}, X_val_split shape: {X_val_split.shape}")

    tfms = [None, [Categorize()]]
    dsets_train = TSDatasets(X_train_split, y_train_split, tfms=tfms, splits=None, inplace=True)
    dls_train = TSDataLoaders.from_dsets(dsets_train.train, dsets_train.train, bs=batch_size,shuffle=False, batch_tfms=[TSStandardize()], num_workers=0)

    c_in = dls_train.vars
    c_out = dls_train.c

    model = CustomCNN(c_in, c_out).to(device)
    learn = Learner(dls_train, model, loss_func=training_loss, metrics=accuracy)
    learn.fit_one_cycle(25, 0.002)

    model.eval()

    classifier = create_art_classifier(model, input_shape=(c_in, X_train_split.shape[2]), nb_classes=c_out, device=device)

    X_val_pieces = split_data(X_val_split, num_pieces=5)

    attacked_X_val_split = apply_attack_decision(X_val_pieces, classifier, attack_decisions)

    def extract_embeddings(X_split, save_path):
        X_split_tensor = torch.tensor(X_split).float().to(device)
        all_embeddings = []
        for i in range(0, len(X_split_tensor), batch_size):
            batch_X = X_split_tensor[i:i+batch_size]
            with torch.no_grad():
                _, batch_embeddings = model(batch_X, return_embeddings=True)
            all_embeddings.append(batch_embeddings.cpu().numpy())
        all_embeddings = np.concatenate(all_embeddings, axis=0)
        np.save(save_path, all_embeddings)

    extract_embeddings(X_train_split, save_path)

    extract_embeddings(attacked_X_val_split, attacked_val_save_path)

dataset_ids = ['Libras', 'RacketSports', 'NATOPS', 'UWaveGestureLibrary', 'Cricket', 'Ering', 'BasicMotions', 'Epilepsy']
hidden_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

attack_decisions = generate_attack_decisions(num_pieces=5)
print(f"Attack decisions for each piece: {attack_decisions}")

train_embeddings_dict = {}
attacked_val_embeddings_dict = {}

for dsid in dataset_ids:
    train_save_path = f'train_embeddings_{dsid}.npy'
    attacked_val_save_path = f'attacked_val_embeddings_{dsid}.npy'
    train_and_attack_validation(dsid, hidden_size, device, attack_decisions, batch_size=16, save_path=train_save_path, attacked_val_save_path=attacked_val_save_path)

    train_embeddings_dict[dsid] = np.load(train_save_path)
    attacked_val_embeddings_dict[dsid] = np.load(attacked_val_save_path)

normalized_train_embeddings_dict = {dsid: normalize(train_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}
normalized_attacked_val_embeddings_dict = {dsid: normalize(attacked_val_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}

for dsid1, dsid2 in itertools.combinations(dataset_ids, 2):

    train_embeddings1_normalized = normalized_train_embeddings_dict[dsid1]
    train_embeddings2_normalized = normalized_train_embeddings_dict[dsid2]
    train_similarity_matrix = cosine_similarity(train_embeddings1_normalized, train_embeddings2_normalized)
    avg_train_similarity = np.mean(train_similarity_matrix)
    print(f"Average train set similarity between {dsid1} and {dsid2}: {avg_train_similarity}")

    attacked_val_embeddings1_normalized = normalized_attacked_val_embeddings_dict[dsid1]
    attacked_val_embeddings2_normalized = normalized_attacked_val_embeddings_dict[dsid2]
    attacked_val_similarity_matrix = cosine_similarity(attacked_val_embeddings1_normalized, attacked_val_embeddings2_normalized)
    avg_attacked_val_similarity = np.mean(attacked_val_similarity_matrix)
    print(f"Average attacked validation set similarity between {dsid1} and {dsid2}: {avg_attacked_val_similarity}")


In [ ]:
#CASE 4

import random
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tsai.all import *
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from art.attacks.evasion import (
    BasicIterativeMethod, DeepFool, CarliniL2Method, MomentumIterativeMethod,
    ElasticNet, AutoProjectedGradientDescent, FastGradientMethod, ZooAttack, BoundaryAttack
)
from art.estimators.classification import PyTorchClassifier

torch.use_deterministic_algorithms(False)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

def initialize_weights(model):
    for name, param in model.named_parameters():
        if 'weight' in name:
            torch.nn.init.kaiming_uniform_(param, nonlinearity='relu')  
        elif 'bias' in name:
            torch.nn.init.zeros_(param)  

class CustomCNN(nn.Module):
    def __init__(self, input_size, output_size, num_filters=64, kernel_size=3, dropout=0.5):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_size)
        self.fc1 = nn.Linear(num_filters, output_size)
        self.dropout = nn.Dropout(dropout)

        initialize_weights(self)

    def forward(self, x, return_embeddings=False):
        x = x.permute(0, 1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.adaptive_max_pool1d(x, 1).squeeze(2)
        x = self.dropout(x)
        final_output = self.fc1(x)

        if return_embeddings:
            embeddings = x
            return final_output, embeddings
        else:
            return final_output

def training_loss(pred, target):
    return F.cross_entropy(pred, target)

def create_art_classifier(model, input_shape, nb_classes, device):
    classifier = PyTorchClassifier(
        model=model,
        loss=training_loss,
        input_shape=input_shape,
        nb_classes=nb_classes,
        optimizer=torch.optim.Adam(model.parameters(), lr=0.002),
        device_type=device.type
    )
    return classifier

def apply_attack(X, classifier, attack_number):
    if attack_number == 0:
        attack = BasicIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 1:
        attack = DeepFool(classifier=classifier)
    elif attack_number == 2:
        attack = CarliniL2Method(classifier=classifier)
    elif attack_number == 3:
        attack = MomentumIterativeMethod(estimator=classifier, eps=0.1)
    elif attack_number == 4:
        attack = ElasticNet(classifier=classifier)
    elif attack_number == 5:
        attack = BoundaryAttack(estimator=classifier, targeted=False)
    elif attack_number == 6:
        attack = FastGradientMethod(estimator=classifier, eps=0.1)
    elif attack_number == 7:
        attack = ZooAttack(classifier=classifier, confidence=0.5, targeted=False)
    elif attack_number == 8:
        attack = AutoProjectedGradientDescent(estimator=classifier, eps=0.1)
    else:
        return X
    return attack.generate(X)

def apply_attack_decision(X_split, classifier, attack_decisions):
    attacked_pieces = []
    for piece, (attack_flag, attack_number) in zip(X_split, attack_decisions):
        if attack_flag == 1:
            attacked_piece = apply_attack(piece, classifier, attack_number)
        else:
            attacked_piece = piece
        attacked_pieces.append(attacked_piece)
    return np.concatenate(attacked_pieces, axis=0)

def split_data(X, num_pieces=5):
    splits = np.array_split(X, num_pieces)
    return splits

def generate_attack_decisions(num_pieces=5):
    attack_decisions = []
    for _ in range(num_pieces):
        attack_flag = random.randint(0, 1)
        attack_number = random.randint(0, 9) if attack_flag == 1 else None
        attack_decisions.append((attack_flag, attack_number))
    return attack_decisions

def train_and_attack_validation(dsid, hidden_size, device, attack_decisions, batch_size=16, save_path='train_embeddings.npy', attacked_val_save_path='attacked_val_embeddings.npy'):

    X_train, y_train, X_test, y_test = get_UCR_data(dsid, return_split=True)
    unique_labels = np.unique(y_train)
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    y_train = np.array([label_map[label] for label in y_train])
    y_test = np.array([label_map[label] for label in y_test])

    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.7, shuffle=False)

    print(f"X_train_split shape: {X_train_split.shape}, X_val_split shape: {X_val_split.shape}")

    tfms = [None, [Categorize()]]
    dsets_train = TSDatasets(X_train_split, y_train_split, tfms=tfms, splits=None, inplace=True)
    dls_train = TSDataLoaders.from_dsets(dsets_train.train, dsets_train.train, bs=batch_size, shuffle=False, batch_tfms=[TSStandardize()], num_workers=0)

    c_in = dls_train.vars
    c_out = dls_train.c

    model = CustomCNN(c_in, c_out).to(device)
    initialize_weights(model)
    learn = Learner(dls_train, model, loss_func=training_loss, metrics=accuracy)
    learn.fit_one_cycle(25, 0.002)

    model.eval()

    classifier = create_art_classifier(model, input_shape=(c_in, X_train_split.shape[2]), nb_classes=c_out, device=device)

    X_val_pieces = split_data(X_val_split, num_pieces=5)

    attacked_X_val_split = apply_attack_decision(X_val_pieces, classifier, attack_decisions)

    def extract_embeddings(X_split, save_path):
        X_split_tensor = torch.tensor(X_split).float().to(device)
        all_embeddings = []
        for i in range(0, len(X_split_tensor), batch_size):
            batch_X = X_split_tensor[i:i+batch_size]
            with torch.no_grad():
                _, batch_embeddings = model(batch_X, return_embeddings=True)
            all_embeddings.append(batch_embeddings.cpu().numpy())
        all_embeddings = np.concatenate(all_embeddings, axis=0)
        np.save(save_path, all_embeddings)

    extract_embeddings(X_train_split, save_path)

    extract_embeddings(attacked_X_val_split, attacked_val_save_path)

dataset_ids = ['Libras', 'RacketSports', 'NATOPS', 'UWaveGestureLibrary', 'Cricket', 'Ering', 'BasicMotions', 'Epilepsy']
hidden_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_embeddings_dict = {}
attacked_val_embeddings_dict = {}

for dsid in dataset_ids:
    attack_decisions = generate_attack_decisions(num_pieces=5)
    print(f"Attack decisions for {dsid}: {attack_decisions}")

    train_save_path = f'train_embeddings_{dsid}.npy'
    attacked_val_save_path = f'attacked_val_embeddings_{dsid}.npy'
    train_and_attack_validation(dsid, hidden_size, device, attack_decisions, batch_size=16, save_path=train_save_path, attacked_val_save_path=attacked_val_save_path)

    train_embeddings_dict[dsid] = np.load(train_save_path)
    attacked_val_embeddings_dict[dsid] = np.load(attacked_val_save_path)

normalized_train_embeddings_dict = {dsid: normalize(train_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}
normalized_attacked_val_embeddings_dict = {dsid: normalize(attacked_val_embeddings_dict[dsid], norm='l2') for dsid in dataset_ids}

for dsid1, dsid2 in itertools.combinations(dataset_ids, 2):

    train_embeddings1_normalized = normalized_train_embeddings_dict[dsid1]
    train_embeddings2_normalized = normalized_train_embeddings_dict[dsid2]
    train_similarity_matrix = cosine_similarity(train_embeddings1_normalized, train_embeddings2_normalized)
    avg_train_similarity = np.mean(train_similarity_matrix)
    print(f"Average train set similarity between {dsid1} and {dsid2}: {avg_train_similarity}")

    attacked_val_embeddings1_normalized = normalized_attacked_val_embeddings_dict[dsid1]
    attacked_val_embeddings2_normalized = normalized_attacked_val_embeddings_dict[dsid2]
    attacked_val_similarity_matrix = cosine_similarity(attacked_val_embeddings1_normalized, attacked_val_embeddings2_normalized)
    avg_attacked_val_similarity = np.mean(attacked_val_similarity_matrix)
    print(f"Average attacked validation set similarity between {dsid1} and {dsid2}: {avg_attacked_val_similarity}")
